In [2]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node d5064bb0-408f-4955-8d7f-ec94c5538314

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import time

%matplotlib inline

In [4]:
test_functions = True

In [5]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [6]:
#function to set the speed of the wheels
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

node.send_set_variables(motors(0, 0))

In [7]:
async def motion_control(start_x, start_y, start_theta, goal_x, goal_y, goal_theta):
    # start_theta and goal_theta to be given in degrees, coordinates in mm
    
    # Controller parameters:
    
    # Kx in s^(-1)
    Kx = 1.3
    # Ky in mm^(-2)
    Ky = 0.000015
    # Ktheta in mm^(-1)
    Ktheta = 0.02
    
    # should be given in mm/s
    v_ref = 200
    # should be given in rad/s
    omega_ref = 0
    
    # Converting degrees to radians
    start_theta *= math.pi/180
    goal_theta *= math.pi/180
    
    pos_current = np.array([start_x, start_y, start_theta])
    pos_ref = np.array([goal_x, goal_y, goal_theta])
    
    # kanayama controller
    while np.any(pos_current) != np.any(pos_ref):
        
        # the error x and y are given in the robot referencial, not the global referencial
        pos_error = np.array([[math.cos(pos_current[2]), math.sin(pos_current[2]), 0],
                             [-math.sin(pos_current[2]), math.cos(pos_current[2]), 0],
                             [0, 0, 1]]).dot(pos_ref - pos_current)
        
        print(pos_error)
        
        # q = [v, omega] easier to translate into motor commands
        q = np.array([v_ref*math.cos(pos_error[2]) + Kx*pos_error[0],
                      omega_ref + v_ref*(Ky*pos_error[1] + Ktheta*math.sin(pos_error[2]))])
        
        print(q)

        [left,right] = move_robot(q)
        await node.set_variables(motors(left, right))
        # time.sleep(1)
        
        # should be pos_current = get_filtered_pos()
        pos_current = pos_ref
        
    node.send_set_variables(motors(0, 0))
    print('out')

In [23]:
def move_robot(q):
    
    # distance between the wheels in mm
    b = 95
    
    # convert to motor speed
    v = 2.817 * q[0]
    omega = 8.8/math.pi * q[1]
    
    left_speed = round(-0.5*omega*b + v)
    print('left:',left_speed)
    right_speed = round(0.5*omega*b + v)
    print('right:',right_speed)
    
    return [left_speed,right_speed]

In [26]:
await motion_control(0,0,0,0,0,90)

# Notes: 
# - as the input for the motors depends on the error, a farther goal would make the robot move faster
# - a pure rotation command does not translate into the robot just rotating but we never have to face this case

[0.         0.         1.57079633]
[1.2246468e-14 4.0000000e+00]
left: -532
right: 532
out


In [24]:
[left,right] = move_robot([0,math.pi/2])
await node.set_variables(motors(left, right))
time.sleep(1)
node.send_set_variables(motors(0, 0))

left: -209
right: 209
